In [14]:
# Library imports.
from pyswmm import Simulation, Links, Output
from swmm.toolkit.shared_enum import LinkAttribute, NodeAttribute, SubcatchAttribute
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import time

# Local imports.
from urbansurge import swmm_model, analysis_tools, file_utils

## Physical system states

In [10]:
# Path to configuration file.
physical_config_path = r"C:\Users\ay434\Box\Research\NSF_SCC\SWMM\Canandaigua\Canandaigua_SWMM\canandaigua_config_physical.yml"

In [11]:
# Create instance of SWMM model.
swmm = swmm_model.SWMM(physical_config_path)

# Configure model.
swmm.configure_model()

Simulation info
Flow Units: CFS
System Units: US
Start Time: 2017-01-01 00:00:00
Start Time: 2017-01-02 00:00:00


In [12]:
# OUTPUT TO DATA FRAME
# ---------------------------------------------------------------------
# Get the node depths.
node_depth_df = swmm.get_node_depth()

# Get node flooding.
node_flood_df = swmm.get_node_flooding()

# Get node inflow.
node_total_inflow_df = swmm.get_node_total_inflow()

# Get link flow.
link_flow_df = swmm.get_link_flow()

# Get link depth.
link_depth_df = swmm.get_link_depth()

# Get link velocity.
link_velocity_df = swmm.get_link_velocity()

# Get precipitation time series.
prcp_df = swmm.get_rainfall_timeseries()

# Join data frames.
physical_df = node_depth_df.merge(node_flood_df, on='datetime')
physical_df = physical_df.merge(node_total_inflow_df, on='datetime')
physical_df = physical_df.merge(link_flow_df, on='datetime')
physical_df = physical_df.merge(link_depth_df, on='datetime')
physical_df = physical_df.merge(link_velocity_df, on='datetime')
physical_df = physical_df.merge(prcp_df, on='datetime')

physical_df.head()

,Depth_node_15,Depth_node_16,Depth_node_18,Depth_node_19,Depth_node_20,Depth_node_12,Depth_node_13,Depth_node_14,Depth_node_17,datetime,...,Velocity_link_1,Velocity_link_2,Velocity_link_5,Velocity_link_6,Velocity_link_7,Velocity_link_8,Velocity_link_9,Velocity_link_10,Velocity_link_11,prcp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-01-01 00:10:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-01-01 00:20:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-01-01 00:30:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-01-01 00:40:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017-01-01 00:50:00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Calibrate fault parameters

In [9]:
# Path to configuration file.
model_config_path = r"C:\Users\ay434\Box\Research\NSF_SCC\SWMM\Canandaigua\Canandaigua_SWMM\canandaigua_config_healthy.yml"

In [13]:
# Create instance of SWMM model.
swmm = swmm_model.SWMM(model_config_path)

# Configure model.
swmm.configure_model()

Simulation info
Flow Units: CFS
System Units: US
Start Time: 2017-01-01 00:00:00
Start Time: 2017-01-02 00:00:00


In [ ]:
# Fault parameter search space. Grid search.
N = 30
silt_depths = np.linspace(0,2,N)
L7_roughness = np.linspace(0,0.05,N)
C9_diameter = np.linspace(1,3,N)

# Flow differences (Q_obs - Q_model)^2.
n_links = 9
diffs = np.zeros(N, N, N, n_links)

# Initial values.
# Get storage depth.
Ds_init = swmm.get_storage_property(21, 'MaxDepth')

# Get conduit offsets.
(in_offset_init, out_offset) = swmm.get_link_offsets(11)

# Get the storage invert elevation.
zi_init = swmm.get_storage_property(21, 'Elev.')

for i, silt_depth in enumerate(silt_depths):
    for j, L7_n in enumerate(L7_roughness):
        for k, C9_d in enumerate(C9_diameter):
            # --------------------------------------------
            # Update silt depth.
            # Update the storage_depth.
            Ds = Ds_init - silt_depth
            swmm.set_storage_property(21, 'MaxDepth', Ds)

            # Update conduit offset.
            in_offset = in_offset_init - silt_depth
            swmm.set_link_offsets(11, (in_offset, out_offset))

            # Update storage invert elevation.
            zi = zi_init + silt_depth
            swmm.set_storage_property(21, 'Elev.', zi)
            # --------------------------------------------
            
            # --------------------------------------------
            # Update the link roughness.
            swmm.set_link_roughness(7, L7_n)
            # --------------------------------------------
            
            # --------------------------------------------
            # Update link diameter.
            swmm.set_link_geometry(9, [C9_d, 0, 0, 0])
            # --------------------------------------------
            
            # --------------------------------------------
            # OUTPUT
            
            # Get link flow.
            link_flow_df = swmm.get_link_flow()
            
            # --------------------------------------------
            
            # --------------------------------------------
            # Compute differences.
            for ci, col in enumerate(link_flow_df.columns):
                if 'link' in col:
                    diffs[i,j,k,ci] = np.sum((link_flow_df[col] - physical_df[col])**2)